<a href="https://colab.research.google.com/github/iamBehzad/Machin-Vision-and-Image-Processing/blob/main/Human_Posture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math as math
protoFile ="pose_deploy_linevec_faster_4_stages.prototxt"
weightsFile = "pose_iter_160000.caffemodel"

nPoints = 15
POSE_PAIRS = [[0,1], [1,2], [2,3], [3,4], [1,5], [5,6], [6,7], [1,14], [14,8], [8,9], [9,10], [14,11], [11,12], [12,13] ]
#POSE_PAIRS = [[11,12], [12,13] ]

cap = cv2.VideoCapture("videos/Burpee240.mp4")
cap.set(cv2.CAP_PROP_POS_FRAMES, 65)

ret, frame = cap.read()
position = ((int) (frame.shape[1]/2 - 268/2), (int) (frame.shape[0]/2 - 36/2))

# Load model and Forward Pass
net = cv2.dnn.readNetFromCaffe(protoFile, weightsFile)


def angle3pt(a, b, c):
    """Counterclockwise angle in degrees by turning from a to c around b
        Returns a float between 0.0 and 360.0"""
    ang = math.degrees(
        math.atan2(c[1]-b[1], c[0]-b[0]) - math.atan2(a[1]-b[1], a[0]-b[0]))
    return ang + 360 if ang < 0 else ang

while True:
    ret, frame = cap.read()
    if not ret:
        break
    blob = cv2.dnn.blobFromImage(frame, 1.0 / 255, (368, 368), (0, 0, 0), swapRB=False, crop=False)
    net.setInput(blob)
    output = net.forward()

    # X and Y Scale
    height, width, _ = frame.shape
    scaleX = width / output.shape[3]
    scaleY = height / output.shape[2]

    # Empty list to store the detected keypoints
    points = []

    # Treshold 
    threshold = 0.1

    for i in range(nPoints):
        # Obtain probability map
        probMap = output[0, i, :, :]
    
        # Find global maxima of the probMap.
        _, prob, _, point = cv2.minMaxLoc(probMap)
    
        # Scale the point to fit on the original image
        x = scaleX * point[0]
        y = scaleY * point[1]

        if prob > threshold : 
            # Add the point to the list if the probability is greater than the threshold
            points.append((int(x), int(y)))
        else :
            points.append(None)
            
    # Draw skeleton
    imSkeleton = frame.copy()
    for pair in POSE_PAIRS:
        partA = pair[0]
        partB = pair[1]

        if points[partA] and points[partB]:
            cv2.line(imSkeleton, points[partA], points[partB], (255, 255,0), 2)
            cv2.circle(imSkeleton, points[partA], 8, (255, 0, 0), thickness=-1, lineType=cv2.FILLED)
            
    if points[11] and points[12] and points[13]:

        theta1=angle3pt(points[11],points[12],points[13])
        theta2=angle3pt(points[12],points[13],(points[13][0]+5,points[13][1]))
        #print("theta=",theta)
        
        if (theta1<120 and theta2<145):
            cv2.putText(imSkeleton,"Sitting", position,cv2.FONT_HERSHEY_SIMPLEX,1,(209, 80, 0, 255), 3) 
        elif (theta1>120 and theta2<145):
            cv2.putText(imSkeleton,"Standing", position,cv2.FONT_HERSHEY_SIMPLEX,1,(209, 80, 0, 255), 3) 
        else:
            cv2.putText(imSkeleton,"Lying down", position,cv2.FONT_HERSHEY_SIMPLEX,1,(209, 80, 0, 255), 3) 
                
    cv2.imshow('image',imSkeleton)
    
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cv2.destroyAllWindows()
cap.release()